# HELP

https://practicaldatascience.co.uk/machine-learning/how-to-use-category-encoders-to-transform-categorical-variables
https://xgboost.readthedocs.io/en/latest/parameter.html

## Load modules and packages

In [ ]:
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn.metrics import roc_auc_score
import random
from sklearn.pipeline import Pipeline
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingRandomSearchCV

## Load data

In [ ]:
random.seed(10) 

from functions import *
# Load data - set index column, decimal point, separator
data = pd.read_csv('hw1_devsample.csv', sep=',',
                   decimal='.', index_col='SK_ID_CURR')

# print time of data being loaded - use strftime
print(f'Data loaded on:   {datetime.datetime.now().strftime(format="%Y-%m-%d %H:%M:%S")}')
data_xgb = data.copy()

data_test = pd.read_csv('hw1_outofsample.csv', sep=',',decimal='.', index_col='SK_ID_CURR')

## Small changes

In [ ]:
target = data_xgb['TARGET']
data_xgb = data_xgb.drop(['MONTH','TIME','DAY','BASE','TARGET'], axis=1)
data_xgb.info()

In [44]:
data_test = data_test.drop(['MONTH','TIME','DAY','BASE'], axis=1)

In [ ]:
cols_pred = data_xgb.columns
cols_pred_cat = [col for col in cols_pred if data_xgb[col].dtype == 'O']
cols_pred_cat

## Split data train and test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_xgb, target, test_size=0.2, random_state=123)

# Test manually

### Test target encoder

In [ ]:
target_encoder = ce.target_encoder.TargetEncoder(verbose=0, 
cols=cols_pred_cat,
#drop_invariant=False, 
return_df=True, handle_missing='value', 
handle_unknown='value',
#, min_samples_leaf=1
#smoothing=1.0
)

In [ ]:
X_train = target_encoder.fit_transform(X_train, y_train)
X_train.info()

In [ ]:
# find columns with infinity values
cols_with_inf = []
for col in X_train.columns:
    if np.any(np.isinf(X_train[col])):
        cols_with_inf.append(col)
        print(f'Column {col} includes infinity values.')

# find columns with negative infinity values
cols_with_neginf = []
for col in X_train.columns:
    if np.any(np.isneginf(X_train[col])):
        cols_with_neginf.append(col)
        print(f'Column {col} includes negative infinity values.')

for col in cols_with_inf:
    X_train[col].replace(np.inf, 9999999, inplace = True)

### Test manually XGBOOST

In [ ]:
model = xgb.XGBClassifier(objective='binary:logistic', booster='gbtree', max_depth = 4, eta = 0.4, gamma = 3)

In [ ]:
print(model)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
X_test = target_encoder.fit_transform(X_test, y_test)
X_test.info()

In [ ]:
pred_test = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, pred_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# Pipeline

In [ ]:
target_encoder = ce.target_encoder.TargetEncoder( 
cols=cols_pred_cat,
#drop_invariant=False, 
return_df=True, handle_missing='value', 
handle_unknown='value'
#, min_samples_leaf=1
#, smoothing=1.0
)

In [ ]:
model = xgb.XGBClassifier(objective='binary:logistic', booster='gbtree')

In [ ]:
pipe = Pipeline([
    ('Mean_Target_Encoding', target_encoder),
    ('model', model)
])

## Halving Random Search CV

In [ ]:
param_grid = {
    'Mean_Target_Encoding__smoothing': [0.5, 1.0, 1.5,2.0],
    'model__max_depth': [2, 4, 6, 8],
    'model__eta' : [0.2,0.3,0.4],
    'model__gamma': [0,1,2,3,4,5]
}


## Apply pipeline to data

In [ ]:
search = HalvingRandomSearchCV(pipe, param_grid, cv = 3,factor=5,random_state=0).fit(X_train, y_train)

In [ ]:
search.best_params_ 

{'model__max_depth': 2,
 'model__gamma': 4,
 'model__eta': 0.4,
 'Mean_Target_Encoding__smoothing': 1.0}

In [ ]:
search.best_score_

In [ ]:
pred_test = search.predict_proba(X_test)

In [49]:
model = xgb.XGBClassifier(objective='binary:logistic', booster='gbtree',max_depth= 2,
 gamma= 4,
 eta= 0.4)

In [50]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False, eta=0.4,
              eval_metric=None, gamma=4, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.400000006, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, ...)

In [59]:
pred_test = model.predict(X_test)
print('AUC',roc_auc_score(y_test, pred_test))

AUC 0.5073414179525336


## Calculate AUC

In [ ]:
pred_test = search.predict(X_test)

In [ ]:
sum(pred_test)

In [ ]:
sum(y_test)

In [ ]:
accuracy = accuracy_score(y_test, pred_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
print('AUC',roc_auc_score(y_test, pred_test))

# Predict on the second dataset

In [45]:
prediction = search.predict(data_test)

In [ ]:
target_encoder = ce.target_encoder.TargetEncoder(verbose=0, 
cols=cols_pred_cat,
#drop_invariant=False, 
return_df=True, handle_missing='value', 
handle_unknown='value'
#, min_samples_leaf=1
#smoothing=1.0
)
data_test = target_encoder.fit_transform(X_train, y_train)
X_train.info()

In [ ]:
prediction = ...

In [46]:
data_saving = pd.DataFrame({'SK_ID_CURR' : data_test.index ,'prediction' : prediction})
data_saving.to_csv('DS2_22_HW3_CADIOU&PAIN.csv',index=False,sep=',')
data_saving.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   SK_ID_CURR  20000 non-null  int64
 1   prediction  20000 non-null  int32
dtypes: int32(1), int64(1)
memory usage: 234.5 KB
